# Карташов Никита Андреевич, гр. 0373

## Теория принятия решений. Вариант 140


### Выполним постановку задачи:

$x_{ij}$ - переменная, равная количеству постоянных работников, которые в $i$-ую(утреннюю/ночную) смену в $j$-ый день недели начали свою рабочую неделю, т.е вышли после выходных;<br>
$x_{ijk}$ - переменная, равная количеству временных работников, которые в $i$-ую(утреннюю/ночную) смену в $j$-ый день недели в $k$-ую(первую/вторую) половину смены были привлечены к работе;<br>
$w_{ijk}$ - переменная, равная необходимому для работы склада количеству работников в $i$-ую(утреннюю/ночную) смену в $j$-ый день недели в $k$-ую(первую/вторую) половину смены;<br>
$c_{r}$ - недельная плата постоянных работников;<br>
$c_{t}$ - недельная плата временных работников;<br>
Множество работающих в текущий день постоянных сотрудников $M = \{1, 4, 5, 6, 7\}$ при $j = 1$; $\{1, 2, 5, 6, 7\}$ при $j = 2$; $\{1, 2, 3, 6, 7\}$ при $j = 3$; $\{1, 2, 3, 4, 7\}$ при $j = 4$; $\{1, 2, 3, 4, 5\}$ при $j = 5$; $\{2, 3, 4, 5, 6\}$ при $j = 6$; $\{3, 4, 5, 6, 7\}$ при $j = 7$.<br>

То есть, например, при $j=1$, т.е в понедельник, множество работающих в текущий день постоянных сотрудников M - это сотрудники, вышедшие в понедельник и в предыдущие четыре дня: воскресенье(7), субботу(6), пятницу(5), четверг(4).

Целевая функция:
$\displaystyle\sum_{i=1}^2\displaystyle\sum_{j=1}^7{c_{r}*x_{ij}} + \displaystyle\sum_{i=1}^2\displaystyle\sum_{j=1}^7\displaystyle\sum_{k=1}^2 {c_{t}*x_{ijk}} \to min$<br>
Ограничения:<br>

$x_{ij} \geq 0$ для всех $i$, для всех $j$<br>

$x_{ijk} \geq 0$ для всех $i$, для всех $j$, для всех $k$<br>

$(\displaystyle\sum_{m \in M}x_{im}) + {x_{ijk}} \geq w_{ijk}$ для всех $i$, для всех $j$(помним, что каждому $j$ соответствует своё множество $M$), и для всех $k$<br>


### Пункт 1.

#### 1.1 Импортируем необходимые для решения модули:


In [172]:
import math
import numpy as np
from cvxopt import matrix, glpk
from prettytable import PrettyTable

#### 1.2 Введём начальные данные:


In [173]:
need = [
    {"morning": [8, 8], "evening": [7, 6]},  # monday
    {"morning": [6, 5], "evening": [4, 5]},  # tuesday
    {"morning": [6, 5], "evening": [4, 4]},  # wednesday
    {"morning": [5, 6], "evening": [6, 7]},  # thursday
    {"morning": [7, 8], "evening": [8, 9]},  # friday
    {"morning": [9, 8], "evening": [7, 6]},  # saturday
    {"morning": [6, 5], "evening": [4, 4]},  # sunday
]
daysToWork = 5
regularMonthPayment = 51200
temporaryHourPayment = 380
weekDays = [
    "Понедельник",
    "Вторник",
    "Среда",
    "Четверг",
    "Пятница",
    "Суббота",
    "Воскресенье",
]

#### 1.3 Выберем, какие переменные $x_{ij}$ суммировать для дневной и ночной смен для каждого дня:


In [174]:
regularPeopleByDay = [
    [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    #
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
]

#### 1.4 Подготовим переменные $x_{ijk}$, отвечающие за выход временных работников:


In [175]:
temporaryPeopleByDay = np.array([])
amount = len(regularPeopleByDay) * 2
for i in range(amount):
    row = np.concatenate(
        (
            np.zeros(i),
            np.array([1]),
            np.zeros(amount - i - 1),
        )
    )
    temporaryPeopleByDay = (
        np.copy(row) if i == 0 else np.vstack((temporaryPeopleByDay, row))
    )


#### 1.5 Соответственно объединим матрицы, полученные в пунктах 1.3 и 1.4, и в каждой строке матрицы получим переменные, отвечающие за постоянных и временных работников, которые должны будут отработать в этот день:


In [176]:
regularAndTemporaryCombined = []
for i in range(len(regularPeopleByDay)):
    row1 = np.concatenate(
        (np.array(regularPeopleByDay[i]), temporaryPeopleByDay[i * 2])
    )
    row2 = np.concatenate(
        (np.array(regularPeopleByDay[i]), temporaryPeopleByDay[i * 2 + 1])
    )
    shift = np.vstack((row1, row2))
    regularAndTemporaryCombined = (
        np.copy(shift) if i == 0 else np.vstack(
            (regularAndTemporaryCombined, shift))
    )

#### 1.6 Сформируем колонку надобности рабочих:


In [177]:
peopleNeed = []
for i in range(len(need)):
    peopleNeed.extend([need[i]["morning"][0], need[i]["morning"][1]])
for i in range(len(need)):
    peopleNeed.extend([need[i]["evening"][0], need[i]["evening"][1]])

#### 1.7 Составим единичную матрицу переменных $x$, чтобы впоследствии задать для этих переменных ограничение неотрицательности:


In [178]:
notNegativeAmount = np.eye(len(regularPeopleByDay) + len(temporaryPeopleByDay))

#### 1.8 Приведём стоимости постоянных и временных рабочих к одной размерности - плате за неделю, и составим строку с платой каждому рабочему:


In [179]:
regularWeekPayment = regularMonthPayment / 4
temporaryWeekPayment = temporaryHourPayment * 4

cost = np.concatenate(
    (
        np.full(len(regularPeopleByDay), regularWeekPayment),
        np.full(len(temporaryPeopleByDay), temporaryWeekPayment),
    )
)


#### 1.9 Составим целевую функцию и запишем весь набор ограничений:


In [180]:
targetFunction = matrix(cost, tc="d").T
inequalityLeft = matrix(
    [
        [matrix(-1 * regularAndTemporaryCombined).T],
        [matrix(-1 * notNegativeAmount).T],
    ],
    tc="d",
).T
inequalityRight = matrix(
    [
        [matrix(-1 * np.array(peopleNeed)).T],
        [matrix(np.zeros(len(notNegativeAmount))).T],
    ],
    tc="d",
).T

#### 1.10 С помощью солвера "glpk" решим задачу по минимизации целевой функции затрат на рабочих с учётом всех ограничений. Принимаем во внимание, что получаемые нами переменные $x_{ij}$ и $x_{ijk}$ должны быть целыми:


In [181]:
(status, x) = glpk.ilp(
    c=targetFunction,
    G=inequalityLeft,
    h=inequalityRight,
    I=set(range(len(targetFunction))),
)
result = np.array([*x.T]).astype(int)

#### 1.11 Создадим функцию для вывода таблицы выхода на работу постоянных и временных рабочих:


In [182]:
def printResult(resultStatus, resultArray):
    print(f"Статус: {resultStatus}")
    if resultStatus == "optimal":
        regularsTable = PrettyTable(
            ["День недели", "7:00 - 15:00", "15:00 - 23:00"])
        for i in range(len(weekDays)):
            regularsTable.add_row(
                [weekDays[i], resultArray[i], resultArray[i + len(weekDays)]]
            )
        print("График выхода постоянных работников:")
        print(regularsTable, end="\n\n")

        temporariesTable = PrettyTable(
            [
                "День недели",
                "7:00 - 11:00",
                "11:00 - 15:00",
                "15:00 - 19:00",
                "19:00 - 23:00",
            ]
        )
        for i in range(len(weekDays)):
            index_morning = len(regularPeopleByDay) + i * 2
            index_evening = len(regularPeopleByDay) + len(weekDays) * 2 + i * 2
            temporariesTable.add_row(
                [
                    weekDays[i],
                    resultArray[index_morning],
                    resultArray[index_morning + 1],
                    resultArray[index_evening],
                    resultArray[index_evening + 1],
                ]
            )
        print("График выхода временных работников:")
        print(temporariesTable, end="\n\n")
    else:
        print("Оптимальное решение не было найдено.")

#### 1.12 Создадим функцию для вывода таблицы общего количества пристуствующих на работе в каждый период времени:


In [183]:
def amountOfWorkersPresent(rows, col):
    workersByHours = np.matmul(rows, col).astype(int)
    workersTable = PrettyTable(
        [
            "День недели",
            "7:00 - 11:00",
            "11:00 - 15:00",
            "15:00 - 19:00",
            "19:00 - 23:00",
        ]
    )
    for i in range(len(weekDays)):
        index_morning = i * 2
        index_evening = len(weekDays) * 2 + i * 2
        workersTable.add_row(
            [
                weekDays[i],
                workersByHours[index_morning],
                workersByHours[index_morning + 1],
                workersByHours[index_evening],
                workersByHours[index_evening + 1],
            ]
        )
    print("Количество работающих в текущий момент сотрудников на каждые 4 часа:")
    print(workersTable)

#### 1.13 Выведем полученные результаты:


In [184]:
printResult(status, result)
amountOfWorkersPresent(regularAndTemporaryCombined, result)

Статус: optimal
График выхода постоянных работников:
+-------------+--------------+---------------+
| День недели | 7:00 - 15:00 | 15:00 - 23:00 |
+-------------+--------------+---------------+
| Понедельник |      0       |       3       |
|   Вторник   |      3       |       0       |
|    Среда    |      1       |       1       |
|   Четверг   |      1       |       2       |
|   Пятница   |      2       |       1       |
|   Суббота   |      1       |       0       |
| Воскресенье |      0       |       0       |
+-------------+--------------+---------------+

График выхода временных работников:
+-------------+--------------+---------------+---------------+---------------+
| День недели | 7:00 - 11:00 | 11:00 - 15:00 | 15:00 - 19:00 | 19:00 - 23:00 |
+-------------+--------------+---------------+---------------+---------------+
| Понедельник |      4       |       4       |       1       |       0       |
|   Вторник   |      0       |       0       |       0       |       1       

### Пункт 2.

#### 2.1 Проанализируем чувствительность построенного плана к возможному увеличению потребности в работниках утром в субботу:


In [185]:
for i in range(6):
    need[5]["morning"][0] += 1
    inequalityRight[10] = -need[5]["morning"][0]
    (status2, x2) = glpk.ilp(
        c=targetFunction,
        G=inequalityLeft,
        h=inequalityRight,
        I=set(range(len(targetFunction))),
    )
    result2 = np.array([*x2.T]).astype(int)
    print(f'Количество необходимых работников в субботу утром: { need[5]["morning"][0] }.')
    printResult(status2, result2)
    amountOfWorkersPresent(regularAndTemporaryCombined, result2)
    print('\n\n')

Количество необходимых работников в субботу утром: 10.
Статус: optimal
График выхода постоянных работников:
+-------------+--------------+---------------+
| День недели | 7:00 - 15:00 | 15:00 - 23:00 |
+-------------+--------------+---------------+
| Понедельник |      0       |       3       |
|   Вторник   |      3       |       0       |
|    Среда    |      1       |       1       |
|   Четверг   |      1       |       2       |
|   Пятница   |      2       |       1       |
|   Суббота   |      1       |       0       |
| Воскресенье |      0       |       0       |
+-------------+--------------+---------------+

График выхода временных работников:
+-------------+--------------+---------------+---------------+---------------+
| День недели | 7:00 - 11:00 | 11:00 - 15:00 | 15:00 - 19:00 | 19:00 - 23:00 |
+-------------+--------------+---------------+---------------+---------------+
| Понедельник |      4       |       4       |       1       |       0       |
|   Вторник   |      0

#### Видим, что при увеличении потребности в работниках в субботу утром увеличивается только количество нанимаемых на это время временных работников. 